示例为一个简单的利用pytorch进行logsitic回归解决二分类问题

In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 载入数据，这里使用的UCI German Credit是UCI的德国信用数据集
data = np.loadtxt('german.data-numeric')

In [7]:
# 共1000行数据，25个特征（其中最后一列为label，值为1、2）
# 进行数据的归一化
r, c = data.shape
for i in range(c-1):
    meanVal = np.mean(data[:, i])
    stdVal = np.std(data[:, i])
    data[:,i] = (data[:,i]-meanVal)/stdVal

In [8]:
# 打乱数据
np.random.shuffle(data)

# 选择训练集和测试集
train_data = data[:900, :c-1]
train_lab = data[:900, c-1]-1
test_data = data[900:, :c-1]
test_lab = data[900:, c-1]-1

In [12]:
# 定义网络架构
class LRNET(torch.nn.Module):
    def __init__(self):
        super(LRNET, self).__init__()
        self.fc1 = torch.nn.Linear(24, 2)
    def forward(self, x):
        x = self.fc1(x)
        out = F.sigmoid(x)
        return out     

In [15]:
# 定义损失函数和优化器
lrnet = LRNET()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lrnet.parameters(), lr=0.01)
EPOCH = 1000

In [17]:
# 将数据转换为tensor
x = torch.from_numpy(train_data).float()
y = torch.from_numpy(train_lab).long()
x_test = torch.from_numpy(test_data).float()
y_test = torch.from_numpy(test_lab).long()

# 开始训练
for epoch in range(EPOCH):
    lrnet.train()   # 指定训练模式
    y_hat = lrnet(x)
    loss = criterion(y_hat, y)     # 损失
    optimizer.zero_grad()    # 梯度清零
    loss.backward()     # 向后传导
    optimizer.step()
    if epoch % 100 == 0:
        lrnet.eval()   # 指定计算模式
        y_pred = lrnet(x_test)
        accu = torch.mean((torch.max(y_pred, 1)[1] == y_test).float())
        print("Epoch:{}, Loss:{:.4f}, Accuracy:{:.2f}".format(epoch, loss, accu))

Epoch:0, Loss:0.6862, Accuracy:0.60
Epoch:100, Loss:0.5493, Accuracy:0.79
Epoch:200, Loss:0.5324, Accuracy:0.79
Epoch:300, Loss:0.5248, Accuracy:0.78
Epoch:400, Loss:0.5195, Accuracy:0.78
Epoch:500, Loss:0.5160, Accuracy:0.78
Epoch:600, Loss:0.5135, Accuracy:0.77
Epoch:700, Loss:0.5115, Accuracy:0.77
Epoch:800, Loss:0.5098, Accuracy:0.77
Epoch:900, Loss:0.5084, Accuracy:0.77
